In [19]:
# LangSmith API를 이용한 LLM 모니터링
import os
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
from typing import Any, Dict, List

from dotenv import load_dotenv

# LangChain 관련 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LangSmith 임포트
from langsmith import Client
from langsmith.run_helpers import traceable

# 환경설정
load_dotenv()

# langsmith 추적 활성화
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'langsmith_prj_001'
print(f"프로젝트 : {os.getenv('LANGCHAIN_PROJECT')}")

llm = ChatOpenAI(model = 'gpt-4o-mini')
prompt = ChatPromptTemplate.from_template('{question}에대해서 간단하게 설명해주세요')
chain = prompt | llm | StrOutputParser()

test_questions = [
    'AI','LangSmith','대한민국'
]
for q in test_questions:
    response = chain.invoke({'question':q})
    print(f'질문 : {q}  답변 : {response}')


# @traceable 로 설정된 함수는 자동으로 추적
@traceable(name=f'custom_qa_function_{os.getenv('LANGCHAIN_PROJECT')}')
def answer_question(question:str)->str:
    prompt = f'다음 질문에 대해서 100자 이내로 요약해서 답변해주세요 : {question}'
    chain = llm | StrOutputParser()
    return chain.invoke(prompt)

result = answer_question('프로그래밍 전문가가 되는 방법 및 가이드')
print(f'answer_question : {result}')

# client 직접사용
client = Client()
# 프로젝트 목록조회
print('프로젝트 목록조회')
project_lists = list(client.list_projects())
for project in project_lists:
    print(project.name)


# 데이터 셋 생성
dataset_name=f"{os.getenv('LANGCHAIN_PROJECT')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description=os.getenv('LANGCHAIN_PROJECT') + '_QA 평가용 데이터셋'
)
# 평가용 예제
examples = [
    {
        "inputs": {"question": "Python이란 무엇인가요?"},
        "outputs": {"answer": "Python은 프로그래밍 언어입니다."}
    },
    {
        "inputs": {"question": "1+1은?"},
        "outputs": {"answer": "2입니다."}
    },
    {
        "inputs": {"question": "AI란?"},
        "outputs": {"answer": "인공지능입니다."}
    }
]
for ex in examples:
    client.create_example(
        inputs=ex['inputs'],
        outputs=ex['outputs'],
        dataset_id=dataset.id
    )
print(f'    {len(examples)}개 예제 추가 완료')

from langsmith.evaluation import evaluate
client = Client()
# 평가모델 정의
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)
#평가 함수 실행
def predict(inputs:str)->Dict[str,str]:
    q = inputs['question']
    result = llm.invoke(f'{q} 간단히 답해줘')
    return {'answer':result.content}
def simple_correctness(run, example):
    """run.outputs 로 모델 답변을 가져오는 방식"""

    gold = example.outputs["answer"]
    pred = run.outputs["answer"]

    score = 1.0 if gold in pred else 0.0

    return {
        "key": "correctness",
        "score": score,
        "comment": f"gold={gold} | pred={pred}"
    }
# 평가실행
results = evaluate(
    predict,
    data=dataset_name,            
    evaluators=[simple_correctness]
)

print('\n평가 결과 요약')
print(results)



# 정리 (테스트 후 삭제)
# client.delete_dataset(dataset_id=dataset.id)
# print(' 데이터셋 삭제완료')

프로젝트 : langsmith_prj_001
질문 : AI  답변 : AI(인공지능)는 컴퓨터 시스템이 인간의 지능을 모방하거나 인간처럼 학습, 추론, 문제 해결, 이해 및 의사결정을 할 수 있도록 하는 기술입니다. AI는 기계 학습, 자연어 처리, 이미지 인식 등 여러 분야에 적용되며, 데이터 분석이나 자동화, 개인화된 추천 시스템 등 다양한 작업을 수행할 수 있습니다. AI는 사람의 작업을 보조하거나 대체하여 효율성을 높이고, 새로운 가능성을 열어주는 역할을 합니다.
질문 : LangSmith  답변 : LangSmith는 자연어 처리(NLP)와 관련된 기술을 활용하여 여러 언어로 된 콘텐츠를 분석하고 관리하는 플랫폼입니다. 보통 기업이나 개발자들이 자연어 모델을 구축하고, 데이터를 라벨링하며, AI 기반의 언어 모델을 개선하는 데 도움을 주는 도구입니다. 이러한 플랫폼은 모델의 성능을 향상시키고, 사용자 경험을 개선하는 데 중요한 역할을 합니다. LangSmith는 대화형 AI, 텍스트 분석, 번역 등 다양한 응용 분야에서 활용될 수 있습니다.
질문 : 대한민국  답변 : 대한민국, 흔히 한국이라고 불리는 나라는 동아시아에 위치한 나라로, 한반도의 남부에 자리잡고 있습니다. 북쪽으로는 북한과 국경을 가집니다. 수도는 서울이며, 경제적, 정치적, 문화적 중심지입니다.

대한민국은 민주 공화국으로, 대통령이 국가의 수장을 맡고 있습니다. 한국은 경제적으로 매우 발전한 나라인데, 정보통신, 자동차, 전자제품, 반도체 등의 산업이 특히 강세를 보이고 있습니다. 삼성, LG, 현대와 같은 글로벌 기업들이 본사를 두고 있습니다.

문화적으로는 전통과 현대가 조화를 이루고 있으며, K-팝, 한국 영화, 한국 드라마 등이 세계적으로 큰 인기를 끌고 있습니다. 또한, 김치, 불고기, 비빔밥 등 다양한 한국 음식도 유명합니다.

대한민국은 교육 수준이 높고, IT 인프라가 잘 구축되어 있어 정보화 사회로서의 특징도 가지고 있습니다. 역사적으로는 일제 강점기와 한국 전쟁을 

3it [00:08,  2.74s/it]


평가 결과 요약
<ExperimentResults fixed-feeling-64>


In [15]:
from pprint import pprint
pprint(results.__dict__)

{'_manager': <langsmith.evaluation._runner._ExperimentManager object at 0x000001858B6CDEE0>,
 '_processing_complete': <threading.Event at 0x1858b6ce3c0: set>,
 '_queue': <queue.Queue object at 0x000001858B6CDAF0>,
 '_results': [{'evaluation_results': {'results': [EvaluationResult(key='correctness', score=0.0, value=None, comment='gold=인공지능입니다. | pred=AI(인공지능)는 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 문제 해결, 의사 결정 등을 수행할 수 있도록 하는 기술입니다. 머신러닝, 자연어 처리, 컴퓨터 비전 등 다양한 분야에서 활용됩니다.', correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('605a47b9-768d-4ed3-8369-b3c411f06299'), target_run_id=None, extra=None)]},
               'example': <class 'langsmith.schemas.Example'>(id=ce35bc1f-8c09-4bb9-a43a-9bd2c53d6a94, dataset_id=d6d02c9c-3be0-4601-9afe-43525682a846, link='https://smith.langchain.com/o/8e08528f-d22a-43e2-ae1c-bf871639e77c/datasets/d6d02c9c-3be0-4601-9afe-43525682a846/e/ce35bc1f-8c09-4bb9-a43a-9bd2c53d6a94'),
               'run': RunTree(id=019ae813-fc45-7669-ab6b-458376d76a6

In [16]:
results_list = results._results

print("\n=== 평가 결과 요약 ===\n")

for i, item in enumerate(results_list, 1):
    ev = item["evaluation_results"]["results"][0]
    ex = item["example"]
    run = item["run"]

    print(f"[예제 {i}]")
    print(f"- 입력 질문: {ex.inputs['question']}")
    print(f"- 정답(gold): {ev.comment.split('|')[0].replace('gold=','').strip()}")
    print(f"- 모델 출력(pred): {ev.comment.split('|')[1].replace('pred=','').strip()}")
    print(f"- 점수: {ev.score}")
    print(f"- 링크: {ex.link}")
    print("-------------------------------------------------\n")


=== 평가 결과 요약 ===

[예제 1]
- 입력 질문: AI란?
- 정답(gold): 인공지능입니다.
- 모델 출력(pred): AI(인공지능)는 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 문제 해결, 의사 결정 등을 수행할 수 있도록 하는 기술입니다. 머신러닝, 자연어 처리, 컴퓨터 비전 등 다양한 분야에서 활용됩니다.
- 점수: 0.0


AttributeError: 'Example' object has no attribute 'link'